In [217]:
import csv
import escher
from escher import Builder
import pandas
from time import time, sleep

import cobra
from cobra.io import load_model, to_json, save_json_model
from cobra import Model, Reaction, Metabolite
from cobra.manipulation import delete_model_genes, knock_out_model_genes, remove_genes

model = load_model("iJN1463")

# The genes to be deleted one at a time
genes_to_delete = ["phaA","phaB","phaC2","benA","benB","benC","benD","phaG"]
gene_ids = ['PP_5003', 'PP_5004', 'PP_5005', 'PP_3161','PP_3162','PP_3163', 'PP_3164', 'PP_1408']

# Single deletion of each gene in my gene list
gene_list = gene_ids

remove_genes(model, gene_list)


In [218]:
# print(model.reactions)

# model.genes.get_by_id("PP_5003")

In [220]:
medium = model.medium
# medium["EX_nh4_e"] = 4.5
medium["EX_ac_e"] = 10.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

# model.medium

# model.optimize()

phaA_reaction = Reaction("phaA")
phaA_reaction.name = 'acetyl-coa to acetoacetyl-coa'
phaA_reaction.subsystem = 'PHB Production'

phaA_reaction.add_metabolites({
    model.metabolites.get_by_id("accoa_c"): -2.0,
    model.metabolites.get_by_id("coa_c"): 1.0,
    model.metabolites.get_by_id("aacoa_c"): 1.0
})

# phaA_reaction.reaction

phaB_reaction = Reaction("phaB")
phaB_reaction.name = 'acetoacetyl-coa to 3-hydroxybutanoyl-coa'
phaB_reaction.subsystem = 'PHB Production'

phaB_reaction.add_metabolites({
    model.metabolites.get_by_id("nadph_c"): -1.0,
    model.metabolites.get_by_id("aacoa_c"): -1.0,
    model.metabolites.get_by_id("nadp_c"): 1.0,
    model.metabolites.get_by_id("3hbcoa_c"): 1.0
})

phaC_reaction = Reaction("PHAP2C4")
phaC_reaction.name = '3-hydroxybutanoyl-coa to PHB'
phaC_reaction.subsystem = 'PHB Production'

phaC_reaction.add_metabolites({
    model.metabolites.get_by_id("3hbcoa_c"): -1.0,
    model.metabolites.get_by_id("bhb_c"): -1.0,
    model.metabolites.get_by_id("PHAg_c"): 1.0,
    model.metabolites.get_by_id("coa_c"): 1.0
})


# print(phaC_reaction.reaction)

model.add_reactions([phaA_reaction, phaB_reaction, phaC_reaction])
# model.add_reactions([phaA_reaction, phaB_reaction])

model.objective = "PHAP2C4"
# model.objective = "phaB"
# model.objective = "BIOMASS_KT2440_WT3"

# biomass_rxn = model.reactions.get_by_id("BIOMASS_KT2440_WT3")

solution = model.optimize()

field_names = ['No', 'Reaction ID', 'Flux Value']

solution.fluxes.to_csv("/Users/neha/Desktop/mit/misti/data/fba/flux_values.csv")

model.optimize()


Ignoring reaction 'phaA' since it already exists.
Ignoring reaction 'phaB' since it already exists.
Ignoring reaction 'PHAP2C4' since it already exists.


,fluxes,reduced_costs
3HAD160,0.00000,0.000000e+00
13DAMPPabcpp,0.00000,5.551115e-17
13DAMPPtex,0.00000,0.000000e+00
15DAPabcpp,0.00000,0.000000e+00
1P2CBXLCYCL,0.00000,-9.230066e-17
...,...,...
4MCAT23DOX,0.00000,-9.454243e-17
4OD,0.00000,-1.275022e-16
phaA,0.00000,2.303884e-16
phaB,3.70406,0.000000e+00


In [158]:
# model.metabolites.accoa_c.summary()

In [159]:
# model.genes.get_by_id("PP_5005")

In [161]:
# from cobra.medium import minimal_medium

# max_growth = model.slim_optimize()
# minimal_medium(model, max_growth)

In [171]:
edited_model = cobra.io.to_json(model)

builder = Builder(
    model_json = edited_model
)

# builder = Builder()

builder 

Builder()

In [204]:
cobra.io.save_json_model(model, "/Users/neha/Desktop/mit/misti/data/fba/test.json")

builder = Builder()

builder 

Builder()